# =============================================================================
# ITS 2122: Python for Data Science & AI - Group Project
# Strategic Growth Analysis for a UK-Based E-Commerce Retailer
# =============================================================================

# Project Overview

This notebook presents a comprehensive analysis of "Unique Gifts Ltd.", a UK-based online retailer specializing in unique giftware. The analysis follows a structured approach to transform raw transactional data into actionable business insights.

## Project Phases:
1. **Data Sanitation and Preprocessing** - Cleaning and preparing the data
2. **Exploratory Data Analysis** - Uncovering patterns and trends
3. **RFM Customer Segmentation** - Advanced customer analytics
4. **Strategic Recommendations** - Business insights and strategies
5. **Data Enrichment** - API integration for currency conversion

## Business Questions Addressed:
- Sales performance and seasonality patterns
- Product portfolio optimization
- Geographic footprint analysis
- Customer segmentation
- Wholesaler vs. retail customer analysis

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Phase 1: Data Sanitation and Preprocessing

In this phase, we clean and prepare the raw dataset for analysis by:
- Handling missing values and duplicates
- Removing cancelled orders and non-product items
- Creating derived features for temporal analysis
- Converting data types appropriately

In [ ]:
# Load the raw dataset
df = pd.read_csv('../data/raw/online_retail.csv', encoding='ISO-8859-1')

# Display basic information
print("Dataset shape:", df.shape)
print("\nDataset info:")
df.info()
print("\nMissing values:")
print(df.isnull().sum())
print("\nBasic statistics:")
df.describe(include='all')

In [ ]:
# Rename columns for consistency
df = df.rename(columns={
    'Invoice': 'InvoiceNo',
    'Price': 'UnitPrice', 
    'Customer ID': 'CustomerID',
    'InvoiceDate': 'InvoiceDate'
})

# Remove duplicates
print("Duplicate count before:", df.duplicated().sum())
df = df.drop_duplicates()
print("Duplicate count after:", df.duplicated().sum())

# Handle cancelled orders (InvoiceNo starting with 'C')
df['InvoiceNo'] = df['InvoiceNo'].astype(str)
cancel_mask = df['InvoiceNo'].str.startswith('C', na=False)
cancelled = df[cancel_mask].copy()
df = df[~cancel_mask]
print(f"Removed {len(cancelled)} cancelled orders")

# Remove rows with non-positive quantity or price
non_pos_mask = (df['Quantity'] <= 0) | (df['UnitPrice'] <= 0)
non_pos_rows = df[non_pos_mask].copy()
df = df[~non_pos_mask]
print(f"Removed {len(non_pos_rows)} rows with non-positive quantity/price")

# Filter out non-product items (postage, adjustments, etc.)
non_product_pattern = r'POST|POSTAGE|ADJUST|BANK|M|CARR|DOT'
mask_nonprod = df['StockCode'].astype(str).str.contains(non_product_pattern, case=False, na=False)
non_prod_rows = df[mask_nonprod].copy()
df = df[~mask_nonprod]
print(f"Removed {len(non_prod_rows)} non-product rows")

# Create TotalPrice column
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Convert InvoiceDate to datetime and extract temporal features
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], dayfirst=True, errors='coerce')
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['DayOfWeek'] = df['InvoiceDate'].dt.day_name()
df['Hour'] = df['InvoiceDate'].dt.hour

# Handle missing CustomerID - create separate dataset for RFM analysis
print(f"Missing CustomerID before: {df['CustomerID'].isnull().sum()}")
df_with_cust = df.dropna(subset=['CustomerID']).copy()
df_with_cust['CustomerID'] = df_with_cust['CustomerID'].astype(int)
print(f"Missing CustomerID after: {df_with_cust['CustomerID'].isnull().sum()}")

# Save cleaned datasets
df.to_csv('../data/processed/online_retail_clean.csv', index=False)
df_with_cust.to_csv('../data/processed/online_retail_clean_with_customerids.csv', index=False)

print("Phase 1 completed. Cleaned datasets saved.")

# Phase 2: Exploratory Data Analysis (EDA)

In this phase, we explore the cleaned data to uncover patterns, trends, and insights that answer the business questions.

In [ ]:
# Load cleaned data
df = pd.read_csv('../data/processed/online_retail_clean.csv')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Display basic information about cleaned data
print("Cleaned dataset shape:", df.shape)
print("\nBasic statistics:")
df.describe()

In [ ]:
# Distribution plots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Quantity distribution
sns.histplot(df['Quantity'], bins=50, kde=False, ax=axes[0])
axes[0].set_title("Distribution of Quantity")
axes[0].set_xlim(0, df['Quantity'].quantile(0.99))  # Remove extreme outliers

# UnitPrice distribution
sns.histplot(df['UnitPrice'], bins=50, kde=False, ax=axes[1])
axes[1].set_title("Distribution of Unit Price")
axes[1].set_xlim(0, df['UnitPrice'].quantile(0.99))  # Remove extreme outliers

# TotalPrice distribution
sns.histplot(df['TotalPrice'], bins=50, kde=False, ax=axes[2])
axes[2].set_title("Distribution of Total Price")
axes[2].set_xlim(0, df['TotalPrice'].quantile(0.99))  # Remove extreme outliers

plt.tight_layout()
plt.show()

In [ ]:
# Temporal analysis - Monthly sales trend
monthly_sales = df.groupby(['Year', 'Month'])['TotalPrice'].sum().reset_index()

plt.figure(figsize=(14, 6))
sns.lineplot(data=monthly_sales, x='Month', y='TotalPrice', hue='Year', marker='o')
plt.title("Monthly Sales Trend (2010-2011)")
plt.ylabel("Total Revenue (£)")
plt.xlabel("Month")
plt.xticks(range(1, 13))
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Sales by day of week
dow_sales = df.groupby('DayOfWeek')['TotalPrice'].sum().reset_index()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_sales['DayOfWeek'] = pd.Categorical(dow_sales['DayOfWeek'], categories=day_order, ordered=True)
dow_sales = dow_sales.sort_values('DayOfWeek')

plt.figure(figsize=(10, 6))
sns.barplot(data=dow_sales, x='DayOfWeek', y='TotalPrice')
plt.title("Sales by Day of Week")
plt.ylabel("Total Revenue (£)")
plt.xlabel("Day of Week")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Geographic analysis - Top countries by revenue
country_sales = df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False).reset_index()
top_10_countries = country_sales.head(10)

plt.figure(figsize=(12, 6))
sns.barplot(data=top_10_countries, x='TotalPrice', y='Country')
plt.title("Top 10 Countries by Revenue")
plt.xlabel("Total Revenue (£)")
plt.ylabel("Country")
plt.show()

# UK vs International revenue
uk_revenue = country_sales[country_sales['Country'] == 'United Kingdom']['TotalPrice'].values[0]
intl_revenue = country_sales[country_sales['Country'] != 'United Kingdom']['TotalPrice'].sum()
total_revenue = uk_revenue + intl_revenue

print(f"UK Revenue: £{uk_revenue:,.2f} ({uk_revenue/total_revenue*100:.1f}%)")
print(f"International Revenue: £{intl_revenue:,.2f} ({intl_revenue/total_revenue*100:.1f}%)")

In [ ]:
# Product performance analysis
# Top 10 products by quantity sold
top_products_qty = df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)

# Top 10 products by revenue
top_products_rev = df.groupby('Description')['TotalPrice'].sum().sort_values(ascending=False).head(10)

# Create comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

sns.barplot(x=top_products_qty.values, y=top_products_qty.index, ax=axes[0])
axes[0].set_title("Top 10 Products by Quantity Sold")
axes[0].set_xlabel("Total Quantity Sold")

sns.barplot(x=top_products_rev.values, y=top_products_rev.index, ax=axes[1])
axes[1].set_title("Top 10 Products by Revenue")
axes[1].set_xlabel("Total Revenue (£)")

plt.tight_layout()
plt.show()

# Phase 3: RFM Customer Segmentation

In this phase, we implement the RFM (Recency, Frequency, Monetary) model to segment customers based on their purchasing behavior.

In [ ]:
# Load dataset with CustomerIDs for RFM analysis
df_rfm = pd.read_csv('online_retail_clean_with_customerids.csv')
df_rfm['InvoiceDate'] = pd.to_datetime(df_rfm['InvoiceDate'])

# Calculate RFM metrics
snapshot_date = df_rfm['InvoiceDate'].max() + pd.Timedelta(days=1)

rfm = df_rfm.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',                                  # Frequency
    'TotalPrice': 'sum'                                      # Monetary
}).reset_index()

rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# Remove any remaining NaN values
rfm = rfm.dropna()

print("RFM metrics calculated for", len(rfm), "customers")
rfm.head()

In [ ]:
# Create RFM scores using quintiles
# For Recency: lower values are better (score 5 = most recent)
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

# For Frequency and Monetary: higher values are better (score 5 = highest)
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

# Create combined RFM segment
rfm['RFM_Segment'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
rfm['RFM_Score'] = rfm[['R_Score', 'F_Score', 'M_Score']].astype(int).sum(axis=1)

# Map to descriptive segments using the refined function
def label_segment(row):
    r, f, m = row['R_Score'], row['F_Score'], row['M_Score']
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    elif f >= 4 and (r >= 3 or m >= 3):
        return 'Loyal Customers'
    elif r >= 4 and (f == 3 or m == 3):
        return 'Potential Loyalists'
    elif r >= 4 and f <= 2 and m <= 2:
        return 'New Customers'
    elif r <= 2 and (f >= 4 or m >= 4):
        return 'At-Risk'
    else:
        return 'Hibernating'

rfm['Segment'] = rfm.apply(label_segment, axis=1)

# Display segment distribution
segment_counts = rfm['Segment'].value_counts()
print("Customer segments with refined labeling:")
print(segment_counts)

# Visualization: Segment Size (Counts)
plt.figure(figsize=(10, 6))
segment_counts.plot(kind='bar')
plt.title("Customer Segments by Count")
plt.xlabel("Segment")
plt.ylabel("Number of Customers")
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.show()

# Visualization: Monetary Distribution (Log Scale)
mon = rfm['Monetary'].clip(lower=0.01)
plt.figure(figsize=(10, 6))
plt.hist(np.log10(mon), bins=40, edgecolor='black', alpha=0.7)
plt.title("Customer Monetary Value Distribution (log10 scale)")
plt.xlabel("log10(Monetary £)")
plt.ylabel("Number of Customers")
plt.grid(alpha=0.3)
plt.show()

# Add analysis of the monetary distribution
print(f"Mean customer monetary value: £{rfm['Monetary'].mean():,.2f}")
print(f"Median customer monetary value: £{rfm['Monetary'].median():,.2f}")
print(f"Top 10% of customers account for {rfm['Monetary'].quantile(0.9) / rfm['Monetary'].sum() * 100:.1f}% of revenue")

# Save RFM results
rfm.to_csv('rfm_segments.csv', index=False)
print("RFM segmentation saved to rfm_segments.csv")

# Phase 4: Strategic Recommendations

In this phase, we analyze the RFM segments and investigate the wholesaler hypothesis to provide actionable business recommendations.

In [ ]:
# Analyze customer spending to identify wholesalers
customer_spend = df_rfm.groupby('CustomerID')['TotalPrice'].sum()

plt.figure(figsize=(10, 6))
sns.histplot(customer_spend, bins=100, kde=True)
plt.xlim(0, 10000)  # Zoom in on most customers
plt.title("Distribution of Customer Spend (£)")
plt.xlabel("Total Spend")
plt.ylabel("Number of Customers")
plt.show()

# Identify wholesalers (threshold: £5000)
wholesaler_threshold = 5000
wholesalers = customer_spend[customer_spend > wholesaler_threshold]
retail_customers = customer_spend[customer_spend <= wholesaler_threshold]

print(f"Wholesalers (spend > £{wholesaler_threshold}): {len(wholesalers)} customers")
print(f"Retail customers: {len(retail_customers)} customers")

# Merge spend data with RFM segments
rfm_with_spend = rfm.merge(customer_spend.rename("TotalSpend"), on="CustomerID")
rfm_with_spend['CustomerType'] = rfm_with_spend['TotalSpend'].apply(
    lambda x: 'Wholesaler' if x > wholesaler_threshold else 'Retail'
)

# Compare behavior between wholesalers and retail customers
behavior_summary = rfm_with_spend.groupby('CustomerType').agg({
    'Monetary': 'mean',
    'Frequency': 'mean', 
    'Recency': 'mean',
    'CustomerID': 'count'
}).rename(columns={
    'Monetary': 'Avg_Spend',
    'Frequency': 'Avg_Orders',
    'Recency': 'Avg_Recency_Days',
    'CustomerID': 'Count'
}).round(2)

print("Behavior comparison:")
print(behavior_summary)

In [ ]:
# Visualize customer types
plt.figure(figsize=(8, 5))
sns.countplot(data=rfm_with_spend, x='CustomerType', order=['Retail', 'Wholesaler'])
plt.title("Retail vs Wholesaler Customer Count")
plt.ylabel("Number of Customers")
plt.show()

# Analyze segments by customer type
segment_by_type = pd.crosstab(rfm_with_spend['Segment'], rfm_with_spend['CustomerType'])
segment_by_type['Total'] = segment_by_type.sum(axis=1)
segment_by_type = segment_by_type.sort_values('Total', ascending=False)

print("Segment distribution by customer type:")
print(segment_by_type)

# Phase 5: Data Enrichment via API Integration

In this phase, we integrate external data through API calls to enrich our dataset with currency conversion information.

In [ ]:
# Get top 100 transactions by revenue
top_100 = df.sort_values('TotalPrice', ascending=False).head(100).copy()

# Fetch exchange rates from API
api_url = "https://api.exchangerate-api.com/v4/latest/GBP"

try:
    response = requests.get(api_url)
    if response.status_code == 200:
        rates = response.json()['rates']
        gbp_to_usd = rates['USD']
        gbp_to_eur = rates['EUR']
        
        print(f"GBP to USD rate: {gbp_to_usd}")
        print(f"GBP to EUR rate: {gbp_to_eur}")
        
        # Add converted price columns
        top_100['TotalPrice_USD'] = top_100['TotalPrice'] * gbp_to_usd
        top_100['TotalPrice_EUR'] = top_100['TotalPrice'] * gbp_to_eur
        
        # Display results
        print("\nTop 5 transactions with currency conversion:")
        display(top_100[['InvoiceNo', 'TotalPrice', 'TotalPrice_USD', 'TotalPrice_EUR']].head())
        
        # Save enriched data
        top_100.to_csv('../data/processed/top_100_transactions_enriched.csv', index=False)
        print("\nEnriched data saved to top_100_transactions_enriched.csv")
    else:
        print(f"API request failed with status code: {response.status_code}")
        # Use fallback rates if API fails
        gbp_to_usd = 1.3
        gbp_to_eur = 1.17
        print("Using fallback exchange rates")
        
except Exception as e:
    print(f"Error fetching exchange rates: {e}")
    # Use fallback rates
    gbp_to_usd = 1.3
    gbp_to_eur = 1.17
    print("Using fallback exchange rates")

# Visualize top transactions in different currencies
plt.figure(figsize=(14, 6))

plt.subplot(1, 3, 1)
sns.barplot(data=top_100.head(10), x='TotalPrice', y='Description')
plt.title("Top 10 Transactions (GBP)")
plt.xlabel("Value (£)")

plt.subplot(1, 3, 2)
sns.barplot(data=top_100.head(10), x='TotalPrice_USD', y='Description')
plt.title("Top 10 Transactions (USD)")
plt.xlabel("Value ($)")

plt.subplot(1, 3, 3)
sns.barplot(data=top_100.head(10), x='TotalPrice_EUR', y='Description')
plt.title("Top 10 Transactions (EUR)")
plt.xlabel("Value (€)")

plt.tight_layout()
plt.show()

# Conclusion

This comprehensive analysis has provided valuable insights into Unique Gifts Ltd.'s business operations. Key findings include:

## Sales Performance:
- Clear seasonal patterns with peaks in November/December (holiday season)
- Thursday is the highest revenue day of the week
- UK market dominates (85%+ of revenue) with significant international opportunities

## Product Portfolio:
- Different products rank differently by quantity vs. revenue
- Both high-volume "bread-and-butter" and high-value "cash cow" products identified

## Customer Segmentation:
- RFM analysis identified distinct customer segments (Champions, Loyal Customers, etc.)
- Wholesalers represent a small but significant portion of high-value customers
- The monetary value distribution shows a long tail with a small number of customers contributing disproportionately to revenue

## Strategic Recommendations:
1. **Targeted Marketing**: Develop segment-specific campaigns
2. **Inventory Optimization**: Align stock with seasonal patterns and product performance
3. **International Expansion**: Focus on top international markets
4. **Wholesaler Strategy**: Develop dedicated account management for high-value wholesalers
5. **Pricing Strategy**: Consider regional pricing based on currency analysis

The data enrichment via API integration demonstrates the value of incorporating external data for international business decisions.

In [ ]:
# Save final notebook outputs
outputs = {
    'monthly_sales': monthly_sales,
    'country_sales': country_sales,
    'top_products_qty': top_products_qty,
    'top_products_rev': top_products_rev,
    'rfm_segments': rfm,
    'wholesaler_analysis': behavior_summary
}

# Create outputs directory
Path('../data/outputs').mkdir(exist_ok=True)

# Save each output
for name, data in outputs.items():
    if hasattr(data, 'to_csv'):
        data.to_csv(f'../data/outputs/{name}.csv', index=False)
    else:
        pd.DataFrame(data).to_csv(f'../data/outputs/{name}.csv', index=False)

print("All analysis outputs saved to outputs/ directory")